In [1]:
import requests
import json
import pandas as pd
import random
import time
from datetime import datetime
from timeit import default_timer as timer
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS, ALL_REGIONS

## Functions

In [2]:
#Better readability for JSON output
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent = 4)
    print(text)
    
def get_trades(tid_start, tid_end, currency):
    
    import time
    if tid_start >= tid_end: #make sure valid starting and ending values provided
        tid = 0
        tid = tid_start #initialize 'tid' to starting value
        tmp_list = [] #temp list to hold dfs from loop
        trades = pd.DataFrame() #final df of trades
        i = 0 #while-loop counter for sleep function
        
        gateway = ApiGateway("https://localbitcoins.com", regions = ["us-west-2"]) #Use ApiGateway from requests_ip_rotator to randomize IP used for requests to avoid rate limiting
        gateway.start() #open gateway, always shutdown as well

        session = requests.Session() #begin session
        session.mount("https://localbitcoins.com", gateway)

        
        try:
            while (tid >= tid_end):
                
                parameters = {
                    'max_tid': tid #passes current value of 'tid' as a parameter into the API request
                }

                url = 'https://localbitcoins.com/bitcoincharts/' + str(currency) + '/trades.json'

                t = session.get(url, params = parameters) #pull last 500 trades, starting with max_tid
                t.raise_for_status() #Returns HTTP error object if error occurs

                if len(t.json()) > 0: #if request returns nothing, avoids an error
                    df = pd.DataFrame(t.json()) #get JSON output into pd DataFrame
                    tmp_list.append(df.iloc[:-1]) #append to temp list, omits last row to avoid repeat obs

                    tid_last = df['tid'].iloc[-1] #gets the last tid of the previous dataset, new starting point for next loop
                    time.sleep(1 + (random.randint(0, 1000) / 1000))#sleep function to avoid API errors, this one moderates time between API requests
                    i += 1 #loop counter
                    
                    if tid != tid_last:
                        tid = tid_last
                    else:
                        raise ValueError('Got furthest back you could')
                    
                    if i%50 == 0:#multiples of 50
                        print(tid)
                        
                        

                    if i%500 == 0: #only for multiples of 500 (get API error around 800 without IP randomization)
                        # time.sleep(1200) #sleep for 20 minutes
                        print('congrats! you have made ' + str(i) + ' API requests, you insane person.')
                        # print(tid)
                else:
                    gateway.shutdown()
                    return None

        except ValueError as err:
            gateway.shutdown()
            print(err)
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades                
                
                
        except Exception as e:
            gateway.shutdown()
            print(t.status_code)
            print(e.response.text)
            print(tid)
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades
        
        except KeyboardInterrupt:
            gateway.shutdown()
            print(tid)
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades
        

            
        
        else:
            gateway.shutdown()
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades

    else:
        print('Please enter a value for tid_start that is greater than tid_end')

## Get list of currencies

In [3]:
altcoin = []
name = []

u = "https://localbitcoins.com/api/currencies/"
c = requests.get(u)
d = c.json()['data']['currencies'] #get to higher level of nesting
symbols = list(d.keys()) #get currency symbols

for currency in d:
    altcoin.append(d[str(currency)]['altcoin'])
    name.append(d[str(currency)]['name'])
    
currencies = pd.DataFrame(zip(symbols, name, altcoin), columns = ['symbol', 'currency', 'altcoin'])

In [5]:
symbols

['ADA',
 'AED',
 'AFN',
 'ALL',
 'AMD',
 'ANG',
 'AOA',
 'ARS',
 'AUD',
 'AWG',
 'AXS',
 'AZN',
 'BAM',
 'BBD',
 'BCH',
 'BDT',
 'BGN',
 'BHD',
 'BIF',
 'BMD',
 'BND',
 'BOB',
 'BRL',
 'BSD',
 'BTN',
 'BWP',
 'BYN',
 'BZD',
 'CAD',
 'CDF',
 'CHF',
 'CLF',
 'CLP',
 'CNH',
 'CNY',
 'COP',
 'CRC',
 'CUC',
 'CUP',
 'CVE',
 'CZK',
 'DASH',
 'DJF',
 'DKK',
 'DOGE',
 'DOP',
 'DOT',
 'DZD',
 'EGP',
 'ERN',
 'ETB',
 'ETH',
 'EUR',
 'FJD',
 'FKP',
 'GBP',
 'GEL',
 'GGP',
 'GHS',
 'GIP',
 'GMD',
 'GNF',
 'GTQ',
 'GYD',
 'HKD',
 'HNL',
 'HRK',
 'HTG',
 'HUF',
 'IDR',
 'ILS',
 'IMP',
 'INR',
 'IQD',
 'IRR',
 'ISK',
 'JEP',
 'JMD',
 'JOD',
 'JPY',
 'KES',
 'KGS',
 'KHR',
 'KMF',
 'KPW',
 'KRW',
 'KWD',
 'KYD',
 'KZT',
 'LAK',
 'LBP',
 'LINK',
 'LKR',
 'LRD',
 'LSL',
 'LTC',
 'LYD',
 'MAD',
 'MDL',
 'MGA',
 'MKD',
 'MMK',
 'MNT',
 'MOP',
 'MRU',
 'MUR',
 'MVR',
 'MWK',
 'MXN',
 'MYR',
 'MZN',
 'NAD',
 'NGN',
 'NIO',
 'NOK',
 'NPR',
 'NZD',
 'OKB',
 'OMR',
 'PAB',
 'PEN',
 'PGK',
 'PHP',
 'PKR',
 'PLN

In [ ]:
currencies_not_included = ['USD'] #Which symbols to not run, need to go back and get AMD and ANG
symbols = list(currencies['symbol'][currencies['altcoin'] == False]) #Remove other cryptocurrencies
symbols = [e for e in symbols if e not in currencies_not_included] #Remove symbols not to run
start_tid = 56510558
end_tid = 6000000 #6000000 corresponds to Feb 2017



for currency in symbols:
    
    start_tid = 56510558
    end_tid = 6000000 #6000000 corresponds to Feb 2017
    print(currency)
    start = timer()

    df = get_trades(start_tid, end_tid, currency)
    path = '../temporary/trades_' + str(currency) + '.csv'
    end = timer()
    time = end - start
    print(time)
    
    if df is not None:
        df.to_csv(path, index = False)

AED
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (0 new).
12207101
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
129.5681436
AFN
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
5.553159500000021
ALL
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
Got furthest back you could
7.790182500000014
AMD
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with f

In [25]:
tmp_list

NameError: name 'tmp_list' is not defined

In [5]:
datetime.fromtimestamp(1664127368)

datetime.datetime(2022, 9, 25, 11, 36, 8)

In [7]:
#See most recent trades
t = requests.get('https://localbitcoins.com/bitcoincharts/BTN/trades.json')
jprint(t.json())

[]


In [9]:
len(t.json())

0